In [2]:
import torch
import torch.nn as nn

# 固定位置编码公式

对于 n 个时间步的词，每一个词对应有维度d 

对于位置编码句子，其行代表时间步，列代表词维度

对于第i行，当列为2j和2j+1时


$$
p_{i, 2j} = sin( { \frac{i}{10000^{{2j/d} } )
$$
$$

p_{i, 2j+1} = cos(\frac{i}{10000^{{2j/d})
$$

# 为什么要使用位置编码
> 答：不像RNN那样子，输入是由先后顺序的，并且拥有记忆单元，因此对应的位置信息在输入的时候就已经被记住了
> 但是, 个人看来，transformer使用批量矩阵乘法，来达到计算时并行的时间，也就是所有的时间步都同时计算了，
> 那就丢失了时间步的信息，为了保持这个信息，因此要使用位置编码
> 

# 位置编码的原理（或者说，位置编码为什么能记忆时间信息）
> 如果通过位置编码的公式来看，位置编码是一个二元函数，其自变量是 i 和 j，也就是位置编码的值和词元位置还有词维度有关
> 并且使用sin和cos可以达到每一个对应的i和j生成的位置编码值都是唯一（这个是不是需要数学证明一下。。。？）
> 
> 


In [5]:
class PositionalEncoding(nn.Module):
    def __init__(self, input_dim, max_len=1000):
        super(PositionalEncoding, self).__init__()
        # 输入的维度为
        # batch_size * num_steps * num_hiddens
        # 则要对每一个时间步进行位置编码
        # 由于是固定位置编码，生成一个固定的位置矩阵就行了
        self.input_dim = input_dim
        self.max_len = max_len
        self.position_matrix = torch.zeros(1, max_len, input_dim)
        X = torch.arange(max_len, dtype=torch.float32).reshape(
            -1, 1   
        ) / torch.pow(10000, ( torch.arange(0, input_dim,dtype=torch.float32) / input_dim ))
        print(X.shape)
        print(self.position_matrix.shape)
        self.position_matrix[:, :, 0::2] = torch.sin(X[:, 0::2])
        self.position_matrix[:, :, 1::2] = torch.cos(X[:, 1::2])
        
        
    def forward(self,X):
        X = X + self.position_matrix[:, :X.shape[1], :].to(device=X.device, dtype=X.dtype)
        return X
    
PositionalEncoding(100)(torch.zeros([64, 128, 100]))

torch.Size([1000, 100])
torch.Size([1, 1000, 100])


tensor([[[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  ...,  1.0000e+00,
           0.0000e+00,  1.0000e+00],
         [ 8.4147e-01,  6.1216e-01,  7.3912e-01,  ...,  1.0000e+00,
           1.2023e-04,  1.0000e+00],
         [ 9.0930e-01, -2.5053e-01,  9.9570e-01,  ...,  1.0000e+00,
           2.4045e-04,  1.0000e+00],
         ...,
         [-6.1604e-01,  6.1846e-01, -2.9353e-01,  ...,  9.9986e-01,
           1.5028e-02,  9.9991e-01],
         [ 3.2999e-01, -2.4278e-01, -9.0428e-01,  ...,  9.9986e-01,
           1.5148e-02,  9.9990e-01],
         [ 9.7263e-01, -9.1570e-01, -9.2466e-01,  ...,  9.9986e-01,
           1.5268e-02,  9.9990e-01]],

        [[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  ...,  1.0000e+00,
           0.0000e+00,  1.0000e+00],
         [ 8.4147e-01,  6.1216e-01,  7.3912e-01,  ...,  1.0000e+00,
           1.2023e-04,  1.0000e+00],
         [ 9.0930e-01, -2.5053e-01,  9.9570e-01,  ...,  1.0000e+00,
           2.4045e-04,  1.0000e+00],
         ...,
         [-6.1604e-01,  6